In [1]:
import torchvision
import torch
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torchvision import transforms
from torchvision.io import read_image
from art.estimators.classification import PyTorchClassifier
# from art.attacks.evasion import ProjectedGradientDescentPyTorch
from torchattacks import PGD
from art.utils import random_sphere
from art.config import ART_NUMPY_DTYPE
import numpy as np
from torchvision.transforms import v2
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights, vgg19 , VGG19_Weights, resnet18, ResNet18_Weights, densenet121,  densenet121, DenseNet121_Weights, vit_b_16, ViT_B_16_Weights, efficientnet_b2, EfficientNet_B2_Weights,  swin_s, Swin_S_Weights, convnext_base, ConvNeXt_Base_Weights, squeezenet1_0, SqueezeNet1_0_Weights

/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/baravrah/.conda/envs/gpu_test/lib/python3.8/

In [2]:
import os
os.environ['SLURM_JOBID']

'570194'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
models = [("vit_b_16",vit_b_16, ViT_B_16_Weights),
          ("efficientnet_b0" , efficientnet_b0, EfficientNet_B0_Weights),
          ("swin_s", swin_s, Swin_S_Weights),
          ("densenet121" , densenet121, DenseNet121_Weights),
          ("resnet18" ,resnet18, ResNet18_Weights)]
            # ("convnext_base", convnext_base,ConvNeXt_Base_Weights)]
         # ("squeezenet1_0",squeezenet1_0, SqueezeNet1_0_Weights)]

In [5]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        if x.shape == (3,224,224):
          x = x.unsqueeze(0)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

In [6]:
import os
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Initialize your custom dataset
# new_dataset = CustomDataset()

In [7]:
def read_image(image_path):
    return Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),])

In [8]:
import pickle
import random
from torch.cuda.amp import autocast
trans = [
    transforms.RandomAffine(degrees=(-2, 2), translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=(224, 224), padding=10),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomAdjustSharpness(sharpness_factor=2,p=1),
    transforms.RandomAutocontrast(1)]


augmentations = transforms.Compose(trans)


In [13]:
e=12.75

In [14]:
import numpy as np
import torch
def dct1(x):
    """
    Discrete Cosine Transform, Type I

    :param x: the input signal
    :return: the DCT-I of the signal over the last dimension
    """
    x_shape = x.shape
    x = x.view(-1, x_shape[-1])

    return torch.fft.fft(torch.cat([x, x.flip([1])[:, 1:-1]], dim=1), 1).real.view(*x_shape)


def idct1(X):
    """
    The inverse of DCT-I, which is just a scaled DCT-I

    Our definition if idct1 is such that idct1(dct1(x)) == x

    :param X: the input signal
    :return: the inverse DCT-I of the signal over the last dimension
    """
    n = X.shape[-1]
    return dct1(X) / (2 * (n - 1))


def dct(x, norm=None):
    """
    Discrete Cosine Transform, Type II (a.k.a. the DCT)

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param x: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the DCT-II of the signal over the last dimension
    """
    x_shape = x.shape
    N = x_shape[-1]
    x = x.contiguous().view(-1, N)

    v = torch.cat([x[:, ::2], x[:, 1::2].flip([1])], dim=1)

    Vc = torch.fft.fft(v)

    k = - torch.arange(N, dtype=x.dtype, device=x.device)[None, :] * np.pi / (2 * N)
    W_r = torch.cos(k)
    W_i = torch.sin(k)

    # V = Vc[:, :, 0] * W_r - Vc[:, :, 1] * W_i
    V = Vc.real * W_r - Vc.imag * W_i
    if norm == 'ortho':
        V[:, 0] /= np.sqrt(N) * 2
        V[:, 1:] /= np.sqrt(N / 2) * 2

    V = 2 * V.view(*x_shape)

    return V


def idct(X, norm=None):
    """
    The inverse to DCT-II, which is a scaled Discrete Cosine Transform, Type III

    Our definition of idct is that idct(dct(x)) == x

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param X: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the inverse DCT-II of the signal over the last dimension
    """

    x_shape = X.shape
    N = x_shape[-1]

    X_v = X.contiguous().view(-1, x_shape[-1]) / 2

    if norm == 'ortho':
        X_v[:, 0] *= np.sqrt(N) * 2
        X_v[:, 1:] *= np.sqrt(N / 2) * 2

    k = torch.arange(x_shape[-1], dtype=X.dtype, device=X.device)[None, :] * np.pi / (2 * N)
    W_r = torch.cos(k)
    W_i = torch.sin(k)

    V_t_r = X_v
    V_t_i = torch.cat([X_v[:, :1] * 0, -X_v.flip([1])[:, :-1]], dim=1)

    V_r = V_t_r * W_r - V_t_i * W_i
    V_i = V_t_r * W_i + V_t_i * W_r

    V = torch.cat([V_r.unsqueeze(2), V_i.unsqueeze(2)], dim=2)
    tmp = torch.complex(real=V[:, :, 0], imag=V[:, :, 1])
    v = torch.fft.ifft(tmp)

    x = v.new_zeros(v.shape)
    x[:, ::2] += v[:, :N - (N // 2)]
    x[:, 1::2] += v.flip([1])[:, :N // 2]

    return x.view(*x_shape).real


def dct_2d(x, norm=None):
    """
    2-dimentional Discrete Cosine Transform, Type II (a.k.a. the DCT)

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param x: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the DCT-II of the signal over the last 2 dimensions
    """
    X1 = dct(x, norm=norm)
    X2 = dct(X1.transpose(-1, -2), norm=norm)
    return X2.transpose(-1, -2)


def idct_2d(X, norm=None):
    """
    The inverse to 2D DCT-II, which is a scaled Discrete Cosine Transform, Type III

    Our definition of idct is that idct_2d(dct_2d(x)) == x

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param X: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the DCT-II of the signal over the last 2 dimensions
    """
    x1 = idct(X, norm=norm)
    x2 = idct(x1.transpose(-1, -2), norm=norm)
    return x2.transpose(-1, -2)


def dct_3d(x, norm=None):
    """
    3-dimentional Discrete Cosine Transform, Type II (a.k.a. the DCT)

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param x: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the DCT-II of the signal over the last 3 dimensions
    """
    X1 = dct(x, norm=norm)
    X2 = dct(X1.transpose(-1, -2), norm=norm)
    X3 = dct(X2.transpose(-1, -3), norm=norm)
    return X3.transpose(-1, -3).transpose(-1, -2)

def idct_3d(X, norm=None):
    """
    The inverse to 3D DCT-II, which is a scaled Discrete Cosine Transform, Type III

    Our definition of idct is that idct_3d(dct_3d(x)) == x

    For the meaning of the parameter `norm`, see:
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.fftpack.dct.html

    :param X: the input signal
    :param norm: the normalization, None or 'ortho'
    :return: the DCT-II of the signal over the last 3 dimensions
    """
    x1 = idct(X, norm=norm)
    x2 = idct(x1.transpose(-1, -2), norm=norm)
    x3 = idct(x2.transpose(-1, -3), norm=norm)
    return x3.transpose(-1, -3).transpose(-1, -2)

In [11]:
"""Implementation of sample attack."""
import os
import torch
from torch.autograd import Variable as V
import torch.nn.functional as F
# from attack_methods import DI,gkern
from torchvision import transforms as T
from tqdm import tqdm
import numpy as np
from PIL import Image
# from Normalize import Normalize
# from loader import ImageNet
# from torch.utils.data import DataLoader
import argparse
# import pretrainedmodels

# parser = argparse.ArgumentParser()
# parser.add_argument('--input_csv', type=str, default='./dataset/images.csv', help='Input directory with images.')
# parser.add_argument('--input_dir', type=str, default='./dataset/images', help='Input directory with images.')
# parser.add_argument('--output_dir', type=str, default='./outputs/', help='Output directory with adversarial images.')
# parser.add_argument('--mean', type=float, default=np.array([0.5, 0.5, 0.5]), help='mean.')
# parser.add_argument('--std', type=float, default=np.array([0.5, 0.5, 0.5]), help='std.')
# parser.add_argument("--max_epsilon", type=float, default=16.0, help="Maximum size of adversarial perturbation.")
# parser.add_argument("--num_iter_set", type=int, default=10, help="Number of iterations.")
# parser.add_argument("--image_width", type=int, default=299, help="Width of each input images.")
# parser.add_argument("--image_height", type=int, default=299, help="Height of each input images.")
# parser.add_argument("--batch_size", type=int, default=10, help="How many images process at one time.")
# parser.add_argument("--momentum", type=float, default=1.0, help="Momentum")
# parser.add_argument("--N", type=int, default=20, help="The number of Spectrum Transformations")
# parser.add_argument("--rho", type=float, default=0.5, help="Tuning factor")
# parser.add_argument("--sigma", type=float, default=16.0, help="Std of random noise")


# opt = parser.parse_args()

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

transforms = T.Compose(
    [T.Resize(299), T.ToTensor()]
)

def clip_by_tensor(t, t_min, t_max):
    """
    clip_by_tensor
    :param t: tensor
    :param t_min: min
    :param t_max: max
    :return: cliped tensor
    """
    result = (t >= t_min).float() * t + (t < t_min).float() * t_min
    result = (result <= t_max).float() * result + (result > t_max).float() * t_max
    return result

def save_image(images,names,output_dir):
    """save the adversarial images"""
    if os.path.exists(output_dir)==False:
        os.makedirs(output_dir)

    for i,name in enumerate(names):
        img = Image.fromarray(images[i].astype('uint8'))
        img.save(output_dir + name)

# T_kernel = gkern(7, 3)

def Spectrum_Simulation_Attack(images, gt, model, min, max):
    """
    The attack algorithm of our proposed Spectrum Simulate Attack
    :param images: the input images
    :param gt: ground-truth
    :param model: substitute model
    :param mix: the mix the clip operation 
    :param max: the max the clip operation
    :return: the adversarial images
    """

    
    image_width = 224
    momentum = 1
    num_iter = 10
    eps = e / 255.0
    alpha = eps / num_iter
    x = images.clone()
    grad = 0
    rho = 0.5
    N = 20
    sigma = 16

    for i in range(num_iter):
        noise = 0
        for n in range(N):
            gauss = torch.randn(x.size()[0], 3, image_width, image_width) * (sigma / 255)
            gauss = gauss.cuda()
            x_dct = dct_2d(x + gauss).cuda()
            mask = (torch.rand_like(x) * 2 * rho + 1 - rho).cuda()
            x_idct = idct_2d(x_dct * mask)
            x_idct = V(x_idct, requires_grad = True)

            # DI-FGSM https://arxiv.org/abs/1803.06978
            # output_v3 = model(DI(x_idct))

            output_v3 = model(x_idct)
            loss = F.cross_entropy(output_v3, gt)
            loss.backward()
            noise += x_idct.grad.data
        noise = noise / N

        x = x + alpha * torch.sign(noise)
        x = clip_by_tensor(x, min, max)
    return x.detach()


In [ ]:
import pickle

batch_size = 100
d = {}
for i , (name_v, model_v, weights_v) in enumerate(models): 
        victim = model_v(weights=weights_v.DEFAULT)
        print("______________________")
        print("Victim: "+ name_v)
    
        S = models.copy()
        S.pop(i)
        victim = model_with_normalization(victim, normalization)
        victim.eval()
        victim = victim.to(device)
        
        with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
            new_data_loader = pickle.load(f)
    
        for j , (name_g,model_g, weights_g) in enumerate(S):
                       row = [0,0,0]
                       print("g: "+ name_g) 
                       g = model_g(weights=weights_g.DEFAULT) 
                       Se = S.copy()
                       Se.pop(j)
                       g = model_with_normalization(g, normalization)
                       g.eval()
                       g = g.to(device)
                       ASR = 0
                       total_l2_score = 0
            
                       for batch , (images, labels) in enumerate(new_data_loader):
                           print(batch)
                           labels =labels.to(device)
                           #PGN
                           images_min = clip_by_tensor(images - e / 255.0, 0.0, 1.0)
                           images_max = clip_by_tensor(images + e / 255.0, 0.0, 1.0)
                           x_p = Spectrum_Simulation_Attack(images, labels, g, images_min, images_max)
                           adv_label_p = victim(x_p).argmax(dim=1)
                           # adv_label_p = adv_label_p.to('cpu')
                           ASR += (labels != adv_label_p).int().sum().item()
                           #print(ASR)
                           l2_score = torch.norm(images - x_p)
                           total_l2_score += l2_score.sum().item()
                        

            
                       d[f'{name_v}_{name_g}_ASR'] = ASR/1000
                       d[f'{name_v}_{name_g}_L2'] = total_l2_score/1000
                       print(ASR)

______________________
Victim: vit_b_16
g: efficientnet_b0
0
1
2
3
4
5
6
7
8
9
419
g: swin_s
0
1
2
3
4
5


In [ ]:
with open('ranking_simba_baselines/SSA.pkl', 'wb') as f:
     pickle.dump(d, f)

In [12]:
x=600

In [ ]:
#SSA

batch_size = 100
for i , (name_v, model_v, weights_v) in enumerate(models): 
    if i==0:
        continue
    if i>1:
        break
    victim = model_v(weights=weights_v.DEFAULT)
    print("______________________")
    print("Victim: "+ name_v)

    S = models.copy()
    S.pop(i)
    victim = model_with_normalization(victim, normalization)
    victim.eval()
    victim = victim.to(device)
    
    with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
        new_data_loader = pickle.load(f)

    new_data_loader = DataLoader(
            new_data_loader.dataset[x:x+100], batch_size=batch_size, shuffle=True
        )
    
    for j , (name_g,model_g, weights_g) in enumerate(S):
                   if j>0:
                       break
                   row = [0,0,0]
                   print("g: "+ name_g) 
                   g = model_g(weights=weights_g.DEFAULT) 
                   Se = S.copy()
                   Se.pop(j)
                   g = model_with_normalization(g, normalization)

                   g.eval()
                   g = g.to(device)
                   ASR_total = 0
                   for batch , (images, labels) in enumerate(new_data_loader):
                       if batch==1:
                            break
                       labels =labels.to(device)
                       augmentations_confidences = []
                       augmentations_success = []

                       print(batch)

                       
                       #SSA
                       images_min = clip_by_tensor(images - e / 255.0, 0.0, 1.0)
                       images_max = clip_by_tensor(images + e / 255.0, 0.0, 1.0)
                       # Spectrum_Simulation_Attack(images, gt, model, images_min, images_max)
                       x_p = Spectrum_Simulation_Attack(images, labels, g, images_min, images_max)
                       adv_label_p = victim(x_p).argmax(dim=1)
                       # adv_label_p = adv_label_p.to('cpu')
                       ASR = (labels != adv_label_p).int().sum().item()
                       print(ASR)
                       row[0] += ASR




                       
                       for _ in tqdm(range(200)):
                            victim = victim.to(device)
                            adv_label_p = victim(x_p).argmax(dim=1)
                            adv_label_p = adv_label_p.to('cpu')
                            # mixing augmentations
                            x_a = augmentations(images)
                            images_min = clip_by_tensor(x_a - e / 255.0, 0.0, 1.0)
                            images_max = clip_by_tensor(x_a + e / 255.0, 0.0, 1.0)
                            x_a = Spectrum_Simulation_Attack(images, labels, g, images_min, images_max)
                            confidance = victim(x_a).softmax(dim=1)
                            # conf = confidance[torch.arange(confidance.size(0)),labels].detach().cpu().numpy()
                            adv_label_a = confidance.argmax(dim=1)


                            torch.cuda.empty_cache()
                            # victim = victim.to('cpu')

                            # average_confidences_p =  np.zeros(batch_size, dtype=np.float32)
                            average_confidences_a =  np.zeros(batch_size, dtype=np.float32)
                            # average_confidences_o =  np.zeros(batch_size, dtype=np.float32)
                            for n , model_s, weights_s in Se:

                                model = model_s(weights=weights_s.DEFAULT)
                                model = model.to(device)
                                model = model_with_normalization(model, normalization)
                                model.eval()


                                outputs_a = model(x_a).softmax(dim=1)
                                confidences_a = outputs_a[torch.arange(outputs_a.size(0)),labels].detach().cpu().numpy()                            
                                average_confidences_a += confidences_a
                                del outputs_a,confidences_a
                                del model


                                torch.cuda.empty_cache()
                            del x_a
                           
                            average_confidences_a /= len(Se)
                            equality_tensor_a = (labels != adv_label_a).int() 



                            augmentations_confidences.append(average_confidences_a)
                            augmentations_success.append(equality_tensor_a)
                            torch.cuda.empty_cache()
                 
                       merged_results_a = [
                                            tuple(
                                                zip(
                                                    np.array(augmentations_confidences)[:, i],
                                                    torch.stack(augmentations_success).cpu().numpy()[:, i]
                                                )
                                            ) 
                                            for i in range(batch_size)
                                        ]


 
                       sorted_results_a = [sorted(image_results, key=lambda x: x[0], reverse=False) for image_results in merged_results_a]

                       best_result = 0
                       random_result = 0 
                       for sublist in sorted_results_a:

                            best_result += sublist[0][1]

                            random_result += random.choice(sublist)[1]


                       row[1]+=random_result
                       row[2]+=best_result

                       print(row) 
                       del images, labels
                       torch.cuda.empty_cache()
                    
                   with open('SSA/'+name_v+'_'+name_g+'_'+str(e)+str(x)+'.pkl', 'wb') as f:
                     pickle.dump(row, f)
                   break

______________________
Victim: efficientnet_b0
g: vit_b_16
0
95


 67%|██████▋   | 134/200 [4:14:32<2:05:10, 113.79s/it]